In [ ]:
import numpy as np
import librosa

In [ ]:


file_path = '/JinsangAffectionTrimmed.mp3'
x, sr = librosa.load(file_path)
onset_frames = librosa.onset.onset_detect(x, sr=sr, wait=1, pre_avg=1, post_avg=1, pre_max=1, post_max=1)
onset_times = librosa.frames_to_time(onset_frames)
# remove extension, .mp3, .wav etc.
save_audio_txt = False#@param {type:"boolean"}

if save_audio_txt:
  file_name_no_extension, _ = os.path.splitext(file_path)
  file_name = f'working_dir/JinsangAffection'
  output_name = file_name_no_extension + '.beatmap.txt'
  with open(output_name, 'wt') as f:
      f.write('\n'.join(['%.4f' % onset_time for onset_time in onset_times]))

In [ ]:
#@markdown Audio compoenent. Step 1: gets array that has time stamps of beats, and magnitudes. Create array whhere index is 
fps = 12 #frames per second
sec_total = 12 # total seconds of audio

t_lin_step = 1 / fps # the amount time represented by index
total_lin_step = sec_total * fps 
t_lin = np.linspace(0, sec_total, num=total_lin_step) # time steps of each frame and weight 

vals, bin_edgs = np.histogram(onset_times, bins=total_lin_step, range=(0,sec_total))

beat_factor_scale = 2.5
beat_factor_mask = ((vals+.5)/2)*beat_factor_scale
print(beat_factor_mask)